In [1]:
from loader import load_reviews_to_list
from text_search import generate_inverted_index, get_tag_frequency_list

In [2]:
path = 'Data\\Restaurants_London_England.tsv'
reviews = load_reviews_to_list(path)

In [3]:
tags, bags = generate_inverted_index(reviews)

NameError: name 's' is not defined

In [46]:
def merge_join(q, tag_list):
    q_ptr = 0
    t_ptr = 0
    results = []
    while q_ptr < len(q) and t_ptr < len(tag_list):
        if q[q_ptr] == tag_list[t_ptr]:
            results.append(t_ptr)
            q_ptr += 1
            t_ptr += 1
        elif q[q_ptr] > tag_list[t_ptr]:
            t_ptr += 1
        else:
            return -1
    return results


def kwSearchIF(text_list, tags_list, bags_list):

    s_text_list = sorted(text_list)

    if_indexes = merge_join(s_text_list, tags_list)
    if len(if_indexes) == 0:
        return "OOPS"
    intersection = set(bags_list[if_indexes[0]])
    if len(if_indexes) > 1:
        for i in range(1, len(if_indexes)):
            intersection = intersection & set(bags_list[if_indexes[i]])

    return list(intersection)
    

In [47]:
q = ['tavern']

reviews_index = kwSearchIF(q, tags, bags)

reviews_index

'OOPS'

In [28]:
for i in reviews_index:
    print(reviews[i]['tags'])

{'review_id': 'Restaurant_Review-g186338-d1015433-Reviews-Costas_Grill-London_England.html',
 'x': 51.508144,
 'y': -0.197733,
 'tags': ['american', 'greek', 'bar', 'grill']}

In [25]:
[i for i in bags[14] if i in bags[73]]

[10303]